# Calculate Number of Tropical Nights

This notebook computes the Tropical Nights index: starting from the daily minimum temperature (2096-2100) TN,
the Tropical Nights index is the number of days where TN > T (T is  a reference temperature, e.g. 20°C)

In [ ]:
# Load packages
import intake
import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

Define Year

In [ ]:
year = "2096"

Load Data Set via Intake

In [ ]:
# Path to catalog descriptor on the DKRZ server
col_url = "/work/ik1017/Catalogs/mistral-cmip6.json"

# Open the catalog with the intake package and name it "col" as short for "collection"
col = intake.open_esm_datastore(col_url)

# This is how we tell intake what data we want

query = dict(
    source_id      = "MPI-ESM1-2-LR", # here we choose Max-Plack Institute's Earth Sytem Model in high resolution
    variable_id    = "tasmin", # temperature at surface, maximum
    table_id       = "day", # daily maximum
    experiment_id  = "ssp585", # what we selected in the drop down menu,e.g. SSP2.4-5 2015-2100
    member_id      = "r10i1p1f1", # "r" realization, "i" initialization, "p" physics, "f" forcing
)

# Intake looks for the query we just defined in the catalog of the CMIP6 data pool at DKRZ
cat = col.search(**query)

# Show query results
cat.df

In [ ]:
src_path = cat.df["path"][4] # this time range contains the selected year

Open Data Set

In [ ]:
ds = xr.open_dataset(src_path)

# Open variable "tasmax" over the whole time range
ds_tasmin = ds["tasmin"]

Look at data set

In [ ]:
ds_tasmin

Select Year

In [ ]:
ds_tasmin_year = ds_tasmin.sel(time=year)

Count the number of tropical nights

In [ ]:
ds_tasmin_year_count = ds_tasmin_year.where(ds_tasmin_year > 293.15).count(dim='time')

Before plotting a cyclic point has to be added, otherwise there will be a gap at the prime meridian

In [ ]:
lon = ds_tasmin_year_count.lon
lat = ds_tasmin_year_count.lat
ds_tasmin_year_count, lon = add_cyclic_point(ds_tasmin_year_count, lon)

Plot data with *cartopy*

In [ ]:
def main():
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())

    plt.contourf(lon, lat, ds_tasmin_year_count, 60,
                transform=ccrs.PlateCarree(),
                cmap='Reds')
    ax.coastlines()
    ax.set_global()
    
    # Add a color bar
    plt.colorbar(ax=ax)
    plt.title('Number of Tropical Nights in Year ' +year)
    plt.show()


if __name__ == '__main__':
    main()